# Find the example case

In [1]:
import os
import sys
import yaml
import xarray as xr
import numpy as np
from glob import glob

In [2]:
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
sys.path.insert(0, os.path.realpath('../libs/'))
import verif_utils as vu

In [4]:
config_name = os.path.realpath('verif_config.yml')

with open(config_name, 'r') as stream:
    conf = yaml.safe_load(stream)

In [5]:
fn_plevel = sorted(
    glob(
        '/glade/derecho/scratch/ksha/CREDIT_data/ERA5_plevel_1deg/upper_subset/ERA5_subset_1deg_6h_*_conserve.zarr'
    )
)

variable_levels = {
    'total_precipitation': None,
}

variable_levels_plevel = {
    'U': [11], # 850 hPa
    'V': [11],
    'specific_total_water': [11,],
    'total_precipitation': None
}

variable_levels_IFS = {
    'u_component_of_wind': [10,], # 925 hPa
    'v_component_of_wind': [10,],
    'specific_humidity': [10,],
    'total_precipitation': None
}

variable_levels_mlevel = {
    'U_mlevel': [14], # 850 hPa
    'V_mlevel': [14],
    'specific_total_water_mlevel': [14,],
    'total_precipitation': None
}

In [6]:
ds_IMERG = xr.open_zarr(
    '/glade/campaign/cisl/aiml/ksha/IMERG_V7/GPM_3B_V07/year_2020_1deg_interp.zarr'
)

In [7]:
ind_6h = 128 - 2 - 4
ind_daily = 31 - 1

lead_ind = 19 - 2 #39
lead_ind_daily = 4 #9

ind_daily_fcst = ind_daily - lead_ind_daily

In [8]:
dict_example = {}

In [9]:
fn_example = fn_plevel[41]
ds_example = xr.open_zarr(fn_example)

U = ds_example['U'].isel(level=10).values[ind_6h, 30:50, 180:280]
V = ds_example['V'].isel(level=10).values[ind_6h, 30:50, 180:280]
Q = ds_example['specific_total_water'].isel(level=10).values[ind_6h, 30:50, 180:280]
print(ds_example['time'].values[ind_6h])

# IMERG tp
TP = ds_IMERG['total_precipitation'].isel(time=ind_daily).values[30:50, 180:280]
print(ds_IMERG['time'].values[ind_daily])

dict_example['ERA5_U'] = U
dict_example['ERA5_V'] = V
dict_example['ERA5_Q'] = Q
dict_example['IMERG_TP'] = TP

2020-01-31T12:00:00.000000000
2020-02-01T00:00:00.000000000


In [10]:
for model_name in ['fuxi_mlevel_dry', 'fuxi_mlevel_physics']:
    filename_OURS = sorted(glob(conf[model_name]['save_loc_gather']+'*2020*.nc'))
    filename_OURS = [fn for fn in filename_OURS if '00Z' in fn]
    
    ds_collect = []
    ds_collect_daily = []
    
    for fn_ours in filename_OURS[:40]:
        # detect 00Z vs 12Z
        ini = int(fn_ours[-6:-4])
        
        ds_ours = xr.open_dataset(fn_ours)
        ds_ours = vu.ds_subset_everything(ds_ours, variable_levels_mlevel)
        ds_collect.append(ds_ours.isel(time=lead_ind))
        
        ds_ours = vu.ds_subset_everything(ds_ours, variable_levels)
        ds_ours_24h = vu.accum_6h_24h(ds_ours, 0)
        ds_ours_24h = ds_ours_24h.isel(time=lead_ind_daily)
        ds_collect_daily.append(ds_ours_24h)
        
    # Combine verif results
    ds_fcst = xr.concat(ds_collect, dim='time')
    ds_fcst_daily = xr.concat(ds_collect_daily, dim='time')
    
    dict_example[f'{model_name}_TP'] = ds_fcst_daily['total_precipitation'].values[ind_daily_fcst, 30:50, 180:280]
    dict_example[f'{model_name}_U'] = ds_fcst['U_mlevel'].values[ind_daily_fcst, 0, 30:50, 180:280]
    dict_example[f'{model_name}_V'] = ds_fcst['V_mlevel'].values[ind_daily_fcst, 0, 30:50, 180:280]
    dict_example[f'{model_name}_Q'] = ds_fcst['specific_total_water_mlevel'].values[ind_daily_fcst, 0, 30:50, 180:280]


In [11]:
model_name = 'fuxi_physics'

filename_OURS = sorted(glob(conf[model_name]['save_loc_gather']+'*2020*.nc'))
filename_OURS = [fn for fn in filename_OURS if '00Z' in fn]

ds_collect = []
ds_collect_daily = []

for fn_ours in filename_OURS[:40]:
    # detect 00Z vs 12Z
    ini = int(fn_ours[-6:-4])
    
    ds_ours = xr.open_dataset(fn_ours)
    ds_ours = vu.ds_subset_everything(ds_ours, variable_levels_plevel)
    ds_collect.append(ds_ours.isel(time=lead_ind))
    
    ds_ours = vu.ds_subset_everything(ds_ours, variable_levels)
    ds_ours_24h = vu.accum_6h_24h(ds_ours, 0)
    ds_ours_24h = ds_ours_24h.isel(time=lead_ind_daily)
    ds_collect_daily.append(ds_ours_24h)
    
# Combine verif results
ds_fcst = xr.concat(ds_collect, dim='time')
ds_fcst_daily = xr.concat(ds_collect_daily, dim='time')

dict_example[f'{model_name}_TP'] = ds_fcst_daily['total_precipitation'].values[ind_daily_fcst, 30:50, 180:280]
dict_example[f'{model_name}_U'] = ds_fcst['U'].values[ind_daily_fcst, 0, 30:50, 180:280]
dict_example[f'{model_name}_V'] = ds_fcst['V'].values[ind_daily_fcst, 0, 30:50, 180:280]
dict_example[f'{model_name}_Q'] = ds_fcst['specific_total_water'].values[ind_daily_fcst, 0, 30:50, 180:280]

In [12]:
# IFS level array([  50,  100,  150,  200,  250,  300,  400,  500,  600,  700,  850,  925, 1000], dtype=int32)

In [13]:
model_name = 'IFS'

filename_OURS = sorted(glob(conf[model_name]['save_loc_gather']+'*2020*.nc'))
filename_OURS = [fn for fn in filename_OURS if '00Z' in fn]

ds_collect = []
ds_collect_daily = []

for fn_ours in filename_OURS[:40]:
    # detect 00Z vs 12Z
    ini = int(fn_ours[-6:-4])
    
    ds_ours = xr.open_dataset(fn_ours)
    ds_ours = vu.ds_subset_everything(ds_ours, variable_levels_IFS)
    ds_collect.append(ds_ours.isel(time=lead_ind))
    
    ds_ours = vu.ds_subset_everything(ds_ours, variable_levels)
    ds_ours_24h = vu.accum_6h_24h(ds_ours, 0)
    ds_ours_24h = ds_ours_24h.isel(time=lead_ind_daily)
    ds_collect_daily.append(ds_ours_24h)
    
# Combine verif results
ds_fcst = xr.concat(ds_collect, dim='time')
ds_fcst_daily = xr.concat(ds_collect_daily, dim='time')

dict_example[f'{model_name}_TP'] = ds_fcst_daily['total_precipitation'].values[ind_daily_fcst, 30:50, 180:280]
dict_example[f'{model_name}_U'] = ds_fcst['u_component_of_wind'].values[ind_daily_fcst, 0, 30:50, 180:280]
dict_example[f'{model_name}_V'] = ds_fcst['v_component_of_wind'].values[ind_daily_fcst, 0, 30:50, 180:280]
dict_example[f'{model_name}_Q'] = ds_fcst['specific_humidity'].values[ind_daily_fcst, 0, 30:50, 180:280]

In [14]:
dict_example.keys()

dict_keys(['ERA5_U', 'ERA5_V', 'ERA5_Q', 'IMERG_TP', 'fuxi_mlevel_dry_TP', 'fuxi_mlevel_dry_U', 'fuxi_mlevel_dry_V', 'fuxi_mlevel_dry_Q', 'fuxi_mlevel_physics_TP', 'fuxi_mlevel_physics_U', 'fuxi_mlevel_physics_V', 'fuxi_mlevel_physics_Q', 'fuxi_physics_TP', 'fuxi_physics_U', 'fuxi_physics_V', 'fuxi_physics_Q', 'IFS_TP', 'IFS_U', 'IFS_V', 'IFS_Q'])

In [19]:
latitude = ds_example['latitude'].values[30:50]
longitude = ds_example['longitude'].values[180:280]
gridx, gridy = np.meshgrid(longitude, latitude)

dict_example['latitude'] = latitude
dict_example['longitude'] = longitude
dict_example['time_str'] = 'Lead time: 5-day; fcst time: 2020-02-01T00:00:00; init time: 2020-01-28T00:00:00'

In [20]:
save_name = '/glade/campaign/cisl/aiml/ksha/CREDIT_cp/VERIF/example_case.npy'

In [21]:
np.save(save_name, dict_example)

In [18]:
# fig, ax = plt.subplots(figsize=(7, 3))

# cf = ax.pcolormesh(gridx, gridy, dict_example[f'{model_name}_TP'], cmap=plt.cm.nipy_spectral_r)
# cbar = fig.colorbar(cf, ax=ax, label='TP')

# skip = (slice(None, None, 3), slice(None, None, 3))
# U_quiv = dict_example[f'{model_name}_U'][skip]
# V_quiv = dict_example[f'{model_name}_V'][skip]
# gridx_quiv = gridx[skip]
# gridy_quiv = gridy[skip]

# # Overlay the horizontal wind vectors as quiver arrows.
# ax.quiver(gridx_quiv, gridy_quiv, U_quiv, V_quiv,
#           color='k', scale=300)

In [19]:
# for i_time in range(1464):
#     if np.max(TP[i_time, ...]) > 0.025:
#         fig, ax = plt.subplots(figsize=(7, 3))
    
#         cf = ax.pcolormesh(gridx, gridy, TP[i_time, ...], cmap=plt.cm.nipy_spectral_r)
#         cbar = fig.colorbar(cf, ax=ax, label='TP')
        
#         skip = (slice(None, None, 3), slice(None, None, 3))
#         U_quiv = U[i_time, ...][skip]
#         V_quiv = V[i_time, ...][skip]
#         gridx_quiv = gridx[skip]
#         gridy_quiv = gridy[skip]
        
#         # Overlay the horizontal wind vectors as quiver arrows.
#         ax.quiver(gridx_quiv, gridy_quiv, U_quiv, V_quiv,
#                   color='k', scale=300)
        
#         ax.set_title(f'time index = {i_time}', fontsize=14)